In [ ]:
# Softmax classifier for guessing minesweeper board position and whether it has a mine or not

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (12,12)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# Clears a square on the minesweeper board.
# If it had a mine, return true
# Otherwise if it has no adjacent mines, recursively run on adjacent squares
# Return false
def clearSquare(board,adjacency,row,col):
    if board[row,col] == 1:
        return True
    if adjacency[row,col] >= 0:
        return False
    n = 0
    for r in range(row-1,row+2):
        for c in range(col-1,col+2):
            if 0 <= r and r < rows and 0 <= c and c < cols:
                n += board[r,c]
    adjacency[row,col] = n
    if n == 0:
        for r in range(row-1,row+2):
            for c in range(col-1,col+2):
                if 0 <= r and r < rows and 0 <= c and c < cols:
                    clearSquare(board,adjacency,r,c)
    return False

In [4]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    clearProbability = r.uniform(0.05,0.5)
    result = np.full(dimensions,-1)
    for index, x in np.ndenumerate(board):
        row,col = index
        if not(x) and result[row,col] == -1 and r.uniform(0,1) < clearProbability:
            clearSquare(board,result,row,col)
    return result

In [5]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    boards = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        validGuesses = np.append(((counts == -1).astype(int) - board).flatten().astype(float),
                                 board.flatten().astype(float))
        validGuessesSum = sum(validGuesses)
        if validGuessesSum > 0:
            # encode counts as one hot
            countsOneHot = np.zeros((counts.size,10))
            countsOneHot[np.arange(counts.size), counts.flatten() + 1] = 1
            batch_xs.append(countsOneHot.flatten())
            batch_ys.append(validGuesses / validGuessesSum)
            boards.append(board)
    return (np.asarray(batch_xs), np.asarray(batch_ys), boards)

In [6]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCountsOneHot = tf.placeholder(tf.float32, [None, size*10], name="mineCountsOneHot")
#mineCountsOneHot = tf.reshape(tf.one_hot(mineCounts+1,10), [-1, size*10])
W = tf.Variable(tf.random_normal([size*10, size*2], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size*2], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [7]:
validGuessAverages = tf.placeholder(tf.float32, [None, size*2], name="validGuessAverages")

In [8]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=validGuessAverages, logits=y))

In [9]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size*2, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows*2, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('loss', cross_entropy)

In [10]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [11]:
# Create session and initialise or restore stuff
savePath = './saves.tf.Mines6/' + str(dimensions) + '/'
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [12]:
tf.global_variables_initializer().run()

In [13]:
# Restore model?
#saver.restore(sess, savePath + "model-2000")

In [14]:
# Train
for iteration in range(10001):
    batch_xs, batch_ys, _ = next_training_batch(1000)
    summary, loss, _ = sess.run([merged, cross_entropy, train_step],
                                  feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
    writer.add_summary(summary, iteration)
    print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    if iteration % 100 == 0:
        save_path = saver.save(sess, savePath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-07 14:18:04: Loss at step 0: 5.672
Model saved in file: ./saves.tf.Mines6/(12, 12)/model-0
2017-11-07 14:18:06: Loss at step 1: 5.6473
2017-11-07 14:18:08: Loss at step 2: 5.63212
2017-11-07 14:18:10: Loss at step 3: 5.61701
2017-11-07 14:18:12: Loss at step 4: 5.61179
2017-11-07 14:18:14: Loss at step 5: 5.6016
2017-11-07 14:18:15: Loss at step 6: 5.59099
2017-11-07 14:18:17: Loss at step 7: 5.57955
2017-11-07 14:18:19: Loss at step 8: 5.57101
2017-11-07 14:18:21: Loss at step 9: 5.56409
2017-11-07 14:18:23: Loss at step 10: 5.55115
2017-11-07 14:18:25: Loss at step 11: 5.53616
2017-11-07 14:18:26: Loss at step 12: 5.53234
2017-11-07 14:18:28: Loss at step 13: 5.52216
2017-11-07 14:18:30: Loss at step 14: 5.51348
2017-11-07 14:18:32: Loss at step 15: 5.50518
2017-11-07 14:18:34: Loss at step 16: 5.49984
2017-11-07 14:18:36: Loss at step 17: 5.49259
2017-11-07 14:18:38: Loss at step 18: 5.48465
2017-11-07 14:18:39: Loss at step 19: 5.4753
2017-11-07 14:18:41: Loss at step 20: 5

2017-11-07 14:23:24: Loss at step 175: 5.00316
2017-11-07 14:23:26: Loss at step 176: 5.01129
2017-11-07 14:23:28: Loss at step 177: 5.02795
2017-11-07 14:23:30: Loss at step 178: 5.00622
2017-11-07 14:23:31: Loss at step 179: 5.00605
2017-11-07 14:23:33: Loss at step 180: 5.01133
2017-11-07 14:23:35: Loss at step 181: 5.00814
2017-11-07 14:23:37: Loss at step 182: 5.00317
2017-11-07 14:23:39: Loss at step 183: 5.02388
2017-11-07 14:23:40: Loss at step 184: 5.00797
2017-11-07 14:23:42: Loss at step 185: 5.01091
2017-11-07 14:23:44: Loss at step 186: 4.99928
2017-11-07 14:23:46: Loss at step 187: 5.00829
2017-11-07 14:23:48: Loss at step 188: 5.00194
2017-11-07 14:23:49: Loss at step 189: 4.99267
2017-11-07 14:23:51: Loss at step 190: 4.994
2017-11-07 14:23:53: Loss at step 191: 4.99309
2017-11-07 14:23:55: Loss at step 192: 4.99052
2017-11-07 14:23:57: Loss at step 193: 4.99223
2017-11-07 14:23:59: Loss at step 194: 4.98473
2017-11-07 14:24:00: Loss at step 195: 4.9845
2017-11-07 14:24

2017-11-07 14:28:39: Loss at step 348: 4.8158
2017-11-07 14:28:41: Loss at step 349: 4.81518
2017-11-07 14:28:43: Loss at step 350: 4.82696
2017-11-07 14:28:45: Loss at step 351: 4.82569
2017-11-07 14:28:47: Loss at step 352: 4.83246
2017-11-07 14:28:49: Loss at step 353: 4.81937
2017-11-07 14:28:50: Loss at step 354: 4.81242
2017-11-07 14:28:52: Loss at step 355: 4.82694
2017-11-07 14:28:54: Loss at step 356: 4.83589
2017-11-07 14:28:56: Loss at step 357: 4.80179
2017-11-07 14:28:58: Loss at step 358: 4.82231
2017-11-07 14:29:00: Loss at step 359: 4.78302
2017-11-07 14:29:01: Loss at step 360: 4.8094
2017-11-07 14:29:03: Loss at step 361: 4.80467
2017-11-07 14:29:05: Loss at step 362: 4.81854
2017-11-07 14:29:07: Loss at step 363: 4.80324
2017-11-07 14:29:09: Loss at step 364: 4.80136
2017-11-07 14:29:11: Loss at step 365: 4.81937
2017-11-07 14:29:12: Loss at step 366: 4.81507
2017-11-07 14:29:14: Loss at step 367: 4.82958
2017-11-07 14:29:16: Loss at step 368: 4.81854
2017-11-07 14:2

2017-11-07 14:33:56: Loss at step 521: 4.73409
2017-11-07 14:33:58: Loss at step 522: 4.71835
2017-11-07 14:34:00: Loss at step 523: 4.71782
2017-11-07 14:34:02: Loss at step 524: 4.73418
2017-11-07 14:34:04: Loss at step 525: 4.72247
2017-11-07 14:34:05: Loss at step 526: 4.72308
2017-11-07 14:34:07: Loss at step 527: 4.71865
2017-11-07 14:34:09: Loss at step 528: 4.73852
2017-11-07 14:34:11: Loss at step 529: 4.72362
2017-11-07 14:34:13: Loss at step 530: 4.70119
2017-11-07 14:34:15: Loss at step 531: 4.7146
2017-11-07 14:34:16: Loss at step 532: 4.72458
2017-11-07 14:34:18: Loss at step 533: 4.7249
2017-11-07 14:34:20: Loss at step 534: 4.72384
2017-11-07 14:34:22: Loss at step 535: 4.73492
2017-11-07 14:34:24: Loss at step 536: 4.7364
2017-11-07 14:34:26: Loss at step 537: 4.70888
2017-11-07 14:34:28: Loss at step 538: 4.68156
2017-11-07 14:34:29: Loss at step 539: 4.74604
2017-11-07 14:34:31: Loss at step 540: 4.71402
2017-11-07 14:34:33: Loss at step 541: 4.69849
2017-11-07 14:34

2017-11-07 14:39:16: Loss at step 695: 4.66016
2017-11-07 14:39:18: Loss at step 696: 4.66473
2017-11-07 14:39:20: Loss at step 697: 4.66157
2017-11-07 14:39:22: Loss at step 698: 4.63704
2017-11-07 14:39:23: Loss at step 699: 4.66629
2017-11-07 14:39:25: Loss at step 700: 4.66673
Model saved in file: ./saves.tf.Mines6/(12, 12)/model-700
2017-11-07 14:39:27: Loss at step 701: 4.68248
2017-11-07 14:39:29: Loss at step 702: 4.66333
2017-11-07 14:39:31: Loss at step 703: 4.6731
2017-11-07 14:39:33: Loss at step 704: 4.65908
2017-11-07 14:39:34: Loss at step 705: 4.66387
2017-11-07 14:39:36: Loss at step 706: 4.67123
2017-11-07 14:39:38: Loss at step 707: 4.66667
2017-11-07 14:39:40: Loss at step 708: 4.67298
2017-11-07 14:39:42: Loss at step 709: 4.67359
2017-11-07 14:39:44: Loss at step 710: 4.66733
2017-11-07 14:39:45: Loss at step 711: 4.6495
2017-11-07 14:39:47: Loss at step 712: 4.67112
2017-11-07 14:39:49: Loss at step 713: 4.66303
2017-11-07 14:39:51: Loss at step 714: 4.64833
2017

...

2017-11-07 15:00:23: Loss at step 1379: 4.56066
2017-11-07 15:00:24: Loss at step 1380: 4.55551
2017-11-07 15:00:26: Loss at step 1381: 4.5413
2017-11-07 15:00:28: Loss at step 1382: 4.55599
2017-11-07 15:00:30: Loss at step 1383: 4.56262
2017-11-07 15:00:32: Loss at step 1384: 4.5454
2017-11-07 15:00:34: Loss at step 1385: 4.56295
2017-11-07 15:00:36: Loss at step 1386: 4.55683
2017-11-07 15:00:37: Loss at step 1387: 4.55416
2017-11-07 15:00:39: Loss at step 1388: 4.56532
2017-11-07 15:00:41: Loss at step 1389: 4.55411
2017-11-07 15:00:43: Loss at step 1390: 4.58103
2017-11-07 15:00:45: Loss at step 1391: 4.58471
2017-11-07 15:00:47: Loss at step 1392: 4.57998
2017-11-07 15:00:49: Loss at step 1393: 4.54398
2017-11-07 15:00:50: Loss at step 1394: 4.54856
2017-11-07 15:00:52: Loss at step 1395: 4.5514
2017-11-07 15:00:54: Loss at step 1396: 4.56017
2017-11-07 15:00:56: Loss at step 1397: 4.56005
2017-11-07 15:00:58: Loss at step 1398: 4.54309
2017-11-07 15:01:00: Loss at step 1399: 4.5

2017-11-07 15:05:37: Loss at step 1548: 4.53782
2017-11-07 15:05:39: Loss at step 1549: 4.54577
2017-11-07 15:05:41: Loss at step 1550: 4.54218
2017-11-07 15:05:43: Loss at step 1551: 4.55123
2017-11-07 15:05:44: Loss at step 1552: 4.54384
2017-11-07 15:05:46: Loss at step 1553: 4.55298
2017-11-07 15:05:48: Loss at step 1554: 4.55117
2017-11-07 15:05:50: Loss at step 1555: 4.56194
2017-11-07 15:05:52: Loss at step 1556: 4.56038
2017-11-07 15:05:54: Loss at step 1557: 4.57043
2017-11-07 15:05:55: Loss at step 1558: 4.54259
2017-11-07 15:05:57: Loss at step 1559: 4.53671
2017-11-07 15:05:59: Loss at step 1560: 4.53204
2017-11-07 15:06:01: Loss at step 1561: 4.53788
2017-11-07 15:06:03: Loss at step 1562: 4.51612
2017-11-07 15:06:05: Loss at step 1563: 4.56079
2017-11-07 15:06:07: Loss at step 1564: 4.5253
2017-11-07 15:06:09: Loss at step 1565: 4.54022
2017-11-07 15:06:11: Loss at step 1566: 4.53001
2017-11-07 15:06:12: Loss at step 1567: 4.52911
2017-11-07 15:06:14: Loss at step 1568: 4

2017-11-07 15:10:52: Loss at step 1717: 4.54209
2017-11-07 15:10:54: Loss at step 1718: 4.54415
2017-11-07 15:10:56: Loss at step 1719: 4.51836
2017-11-07 15:10:58: Loss at step 1720: 4.53819
2017-11-07 15:10:59: Loss at step 1721: 4.54583
2017-11-07 15:11:01: Loss at step 1722: 4.53631
2017-11-07 15:11:03: Loss at step 1723: 4.53366
2017-11-07 15:11:05: Loss at step 1724: 4.55053
2017-11-07 15:11:07: Loss at step 1725: 4.52571
2017-11-07 15:11:09: Loss at step 1726: 4.54919
2017-11-07 15:11:11: Loss at step 1727: 4.52558
2017-11-07 15:11:12: Loss at step 1728: 4.56108
2017-11-07 15:11:14: Loss at step 1729: 4.51824
2017-11-07 15:11:16: Loss at step 1730: 4.54
2017-11-07 15:11:18: Loss at step 1731: 4.52282
2017-11-07 15:11:20: Loss at step 1732: 4.55732
2017-11-07 15:11:22: Loss at step 1733: 4.53849
2017-11-07 15:11:24: Loss at step 1734: 4.53138
2017-11-07 15:11:25: Loss at step 1735: 4.54437
2017-11-07 15:11:27: Loss at step 1736: 4.55434
2017-11-07 15:11:29: Loss at step 1737: 4.5

2017-11-07 15:16:09: Loss at step 1887: 4.53397
2017-11-07 15:16:11: Loss at step 1888: 4.54356
2017-11-07 15:16:13: Loss at step 1889: 4.53251
2017-11-07 15:16:15: Loss at step 1890: 4.54394
2017-11-07 15:16:17: Loss at step 1891: 4.54392
2017-11-07 15:16:19: Loss at step 1892: 4.5321
2017-11-07 15:16:21: Loss at step 1893: 4.5389
2017-11-07 15:16:22: Loss at step 1894: 4.53421
2017-11-07 15:16:24: Loss at step 1895: 4.51713
2017-11-07 15:16:26: Loss at step 1896: 4.53037
2017-11-07 15:16:28: Loss at step 1897: 4.53202
2017-11-07 15:16:30: Loss at step 1898: 4.53713
2017-11-07 15:16:32: Loss at step 1899: 4.53041
2017-11-07 15:16:34: Loss at step 1900: 4.53872
Model saved in file: ./saves.tf.Mines6/(12, 12)/model-1900
2017-11-07 15:16:36: Loss at step 1901: 4.53622
2017-11-07 15:16:37: Loss at step 1902: 4.54622
2017-11-07 15:16:39: Loss at step 1903: 4.53742
2017-11-07 15:16:41: Loss at step 1904: 4.51822
2017-11-07 15:16:43: Loss at step 1905: 4.5253
2017-11-07 15:16:45: Loss at ste

KeyboardInterrupt: 

In [15]:
# Test trained model on larger batch size
batch_xs, batch_ys, _ = next_training_batch(10000)
print(sess.run(cross_entropy, feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys}))

4.52306


In [16]:
# Run a test
batchSize = 10000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
frees = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  10000
240


In [ ]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys, _ = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: batch_xs, validGuessAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

In [ ]:
#batch_xs = [[-1,1,-1,0,0,0,-1,-1,-1,1,1,1,-1,-1,1,2,2,1,1,-1,2,1,1,-1,-1,2,2,-1,-1,2,-1,1,1,0,-1,-1,2,-1,-1,4,-1,2,-1,1,2,-1,1,0,1,2,-1,3,2,2,1,-1,2,-1,1,0,0,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,0,0,3,-1,4,1,2,-1,1,-1,-1,0,0,0,2,-1,-1,-1,2,-1,1,0,0,0,-1,1,2,-1,2,1,2,2,3,3,2,-1,-1,1,-1,1,-1,0,1,2,-1,-1,-1,1,1,1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,1,-1,-1,-1]]
batch_xs0 = [-1] * (size)
batch_xs0[0] = 1
batch_xs0[1] = 1
batch_xs0[cols] = 1

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCountsOneHot: [batch_xs0]})
bestSquares = [pred.argmax() for pred in predictions]

print(bestSquares[0] // cols, bestSquares[0] % cols)

In [17]:
np.save("./W", sess.run(W))

In [18]:
np.save("./b", sess.run(b))

In [19]:
np.savez("./model", sess.run([W,b]))